In [1]:
import os
import pickle as pkl
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd
import seaborn as sns

: 

In [2]:
print(torch.__version__)

2.5.1+cu124


In [3]:
def count(file_path):
    with open(file_path, 'rb') as f:
       return  pkl.load(f)

In [13]:
train= count("/hkfs/home/project/hk-project-pai00011/st_st177261/Knowledge-Graph-project1/data/FB237/train.pickle")
test= count("/hkfs/home/project/hk-project-pai00011/st_st177261/Knowledge-Graph-project1/data/FB237/test.pickle")
valid = count("/hkfs/home/project/hk-project-pai00011/st_st177261/Knowledge-Graph-project1/data/FB237/valid.pickle")

In [14]:
print(train.shape)
print(test.shape)
print(valid.shape)

(272115, 3)
(20466, 3)
(17535, 3)


In [15]:
concatenated = np.concatenate([train, valid, test], axis=0)
print(concatenated.shape)

(310116, 3)


In [18]:
relations_train = train[:, 1]  # Extract the relation IDs (assuming triples are [head, relation, tail])
unique_relations_train, counts_train = np.unique(relations_train, return_counts=True)
# For the valid split
relations_valid = valid[:, 1]
unique_relations_valid, counts_valid = np.unique(relations_valid, return_counts=True)

# For the test split
relations_test = test[:, 1]
unique_relations_test, counts_test = np.unique(relations_test, return_counts=True)

relations_con = concatenated[:,1]
unique_relations_con, counts_con = np.unique(relations_con, return_counts=True)

In [19]:
print("Relation ID  ->  Count")
for rel_id, count in zip(unique_relations_con, counts_con):
    print(f"{rel_id}  ->  {count}")

# Number of unique relations in train split
print("Number of unique relations:", len(unique_relations_con))

Relation ID  ->  Count
0  ->  574
1  ->  181
2  ->  16391
3  ->  2643
4  ->  129
5  ->  726
6  ->  45
7  ->  113
8  ->  1179
9  ->  936
10  ->  121
11  ->  1310
12  ->  838
13  ->  6483
14  ->  128
15  ->  136
16  ->  121
17  ->  263
18  ->  128
19  ->  2321
20  ->  168
21  ->  379
22  ->  391
23  ->  141
24  ->  8491
25  ->  296
26  ->  160
27  ->  369
28  ->  248
29  ->  541
30  ->  313
31  ->  563
32  ->  966
33  ->  2157
34  ->  5834
35  ->  3536
36  ->  154
37  ->  1417
38  ->  233
39  ->  194
40  ->  160
41  ->  149
42  ->  202
43  ->  726
44  ->  458
45  ->  184
46  ->  3040
47  ->  2901
48  ->  145
49  ->  194
50  ->  197
51  ->  779
52  ->  490
53  ->  241
54  ->  609
55  ->  2040
56  ->  7134
57  ->  495
58  ->  851
59  ->  260
60  ->  809
61  ->  109
62  ->  2227
63  ->  499
64  ->  1599
65  ->  2711
66  ->  3887
67  ->  2744
68  ->  191
69  ->  129
70  ->  536
71  ->  685
72  ->  821
73  ->  320
74  ->  1269
75  ->  2622
76  ->  397
77  ->  460
78  ->  162
79  ->  426
80  -

In [31]:
def create_ood_split(all_triples, test_fraction=0.15, seed=42):
    """
    Creates an out-of-distribution split (train/test) from all_triples
    with constraints that no (h,r) or (r,t) pair in test appear in train.
    Then carves out a small valid set from train.

    :param all_triples: List or array of (h, r, t)
    :param test_fraction: Desired fraction of total data in test (e.g., 0.2)
    :param seed: Random seed for shuffling
    :return: train_set, valid_set, test_set
    """
    np.random.seed(seed)
    all_triples = np.random.permutation(all_triples)  # shuffle

    desired_test_size = int(test_fraction * len(all_triples))
    
    used_hr = set()
    used_rt = set()
    test_set = []
    train_set = []
    relations_in_test = set()
    
    # 1) Build train/test
    for (h, r, t) in all_triples:
        can_go_to_test = ((h, r) not in used_hr) and ((r, t) not in used_rt)
        need_for_relation = (r not in relations_in_test)
        not_full_yet = (len(test_set) < desired_test_size)

        if can_go_to_test and (need_for_relation or not_full_yet):
            test_set.append((h, r, t))
            relations_in_test.add(r)
        else:
            train_set.append((h, r, t))
            used_hr.add((h, r))
            used_rt.add((r, t))
    
    # 2) Ensure all relations appear at least once in train as well
    train_relations = set(tr[1] for tr in train_set)
    all_relations = set(tr[1] for tr in all_triples)
    missing_in_train = all_relations - train_relations
    
    # 3) Carve out a valid set from the train set.
    valid_fraction = 0.1
    valid_size = int(valid_fraction * len(train_set))
    np.random.shuffle(train_set)
    valid_set = train_set[:valid_size]
    train_set = train_set[valid_size:]
    
    return np.vstack(train_set), np.vstack(valid_set), np.vstack(test_set)

In [32]:
train_set, valid_set, test_set = create_ood_split(concatenated)

In [33]:
print(len(train_set))
print(len(test_set))
print(len(valid_set))

237240
46517
26359


In [34]:
relations_train = train_set[:,1]
unique_relations_train, counts_train = np.unique(relations_train, return_counts=True)
print("Relation ID  ->  Count")
for rel_id, count in zip(unique_relations_train, counts_train):
    print(f"{rel_id}  ->  {count}")

# Number of unique relations in train split
print("Number of unique relations:", len(unique_relations_train))

Relation ID  ->  Count
0  ->  447
1  ->  135
2  ->  12585
3  ->  2021
4  ->  103
5  ->  560
6  ->  31
7  ->  82
8  ->  917
9  ->  726
10  ->  93
11  ->  1012
12  ->  646
13  ->  4957
14  ->  97
15  ->  105
16  ->  94
17  ->  178
18  ->  94
19  ->  1751
20  ->  131
21  ->  276
22  ->  302
23  ->  100
24  ->  6498
25  ->  229
26  ->  127
27  ->  285
28  ->  187
29  ->  407
30  ->  237
31  ->  436
32  ->  741
33  ->  1643
34  ->  4469
35  ->  2740
36  ->  106
37  ->  1083
38  ->  176
39  ->  147
40  ->  130
41  ->  111
42  ->  158
43  ->  560
44  ->  351
45  ->  138
46  ->  2351
47  ->  2180
48  ->  113
49  ->  157
50  ->  149
51  ->  594
52  ->  352
53  ->  191
54  ->  451
55  ->  1567
56  ->  5444
57  ->  383
58  ->  669
59  ->  200
60  ->  646
61  ->  79
62  ->  1712
63  ->  399
64  ->  1233
65  ->  2070
66  ->  2912
67  ->  2091
68  ->  143
69  ->  91
70  ->  416
71  ->  517
72  ->  621
73  ->  245
74  ->  982
75  ->  2009
76  ->  308
77  ->  365
78  ->  127
79  ->  343
80  ->  179
81

In [35]:
relations_test = test_set[:,1]
unique_relations_test, counts_test = np.unique(relations_test, return_counts=True)
print("Relation ID  ->  Count")
for rel_id, count in zip(unique_relations_test, counts_test):
    print(f"{rel_id}  ->  {count}")

# Number of unique relations in train split
print("Number of unique relations:", len(unique_relations_test))

Relation ID  ->  Count
0  ->  75
1  ->  29
2  ->  2466
3  ->  390
4  ->  19
5  ->  97
6  ->  7
7  ->  21
8  ->  183
9  ->  125
10  ->  17
11  ->  184
12  ->  127
13  ->  1003
14  ->  20
15  ->  19
16  ->  20
17  ->  49
18  ->  22
19  ->  366
20  ->  22
21  ->  53
22  ->  52
23  ->  26
24  ->  1259
25  ->  53
26  ->  20
27  ->  50
28  ->  33
29  ->  78
30  ->  44
31  ->  82
32  ->  151
33  ->  316
34  ->  864
35  ->  515
36  ->  34
37  ->  238
38  ->  33
39  ->  33
40  ->  19
41  ->  32
42  ->  24
43  ->  117
44  ->  58
45  ->  28
46  ->  441
47  ->  461
48  ->  25
49  ->  20
50  ->  24
51  ->  119
52  ->  84
53  ->  30
54  ->  90
55  ->  306
56  ->  1105
57  ->  65
58  ->  127
59  ->  42
60  ->  104
61  ->  20
62  ->  324
63  ->  67
64  ->  224
65  ->  396
66  ->  618
67  ->  406
68  ->  28
69  ->  26
70  ->  81
71  ->  105
72  ->  131
73  ->  51
74  ->  190
75  ->  370
76  ->  60
77  ->  54
78  ->  20
79  ->  52
80  ->  36
81  ->  151
82  ->  28
83  ->  117
84  ->  19
85  ->  23
86  -

In [37]:
relations_valid = valid_set[:,1]
unique_relations_valid, counts_valid = np.unique(relations_valid, return_counts=True)
print("Relation ID  ->  Count")
for rel_id, count in zip(unique_relations_valid, counts_valid):
    print(f"{rel_id}  ->  {count}")

# Number of unique relations in train split
print("Number of unique relations:", len(unique_relations_valid))

Relation ID  ->  Count
0  ->  52
1  ->  17
2  ->  1340
3  ->  232
4  ->  7
5  ->  69
6  ->  7
7  ->  10
8  ->  79
9  ->  85
10  ->  11
11  ->  114
12  ->  65
13  ->  523
14  ->  11
15  ->  12
16  ->  7
17  ->  36
18  ->  12
19  ->  204
20  ->  15
21  ->  50
22  ->  37
23  ->  15
24  ->  734
25  ->  14
26  ->  13
27  ->  34
28  ->  28
29  ->  56
30  ->  32
31  ->  45
32  ->  74
33  ->  198
34  ->  501
35  ->  281
36  ->  14
37  ->  96
38  ->  24
39  ->  14
40  ->  11
41  ->  6
42  ->  20
43  ->  49
44  ->  49
45  ->  18
46  ->  248
47  ->  260
48  ->  7
49  ->  17
50  ->  24
51  ->  66
52  ->  54
53  ->  20
54  ->  68
55  ->  167
56  ->  585
57  ->  47
58  ->  55
59  ->  18
60  ->  59
61  ->  10
62  ->  191
63  ->  33
64  ->  142
65  ->  245
66  ->  357
67  ->  247
68  ->  20
69  ->  12
70  ->  39
71  ->  63
72  ->  69
73  ->  24
74  ->  97
75  ->  243
76  ->  29
77  ->  41
78  ->  15
79  ->  31
80  ->  16
81  ->  90
82  ->  15
83  ->  67
84  ->  10
85  ->  12
86  ->  100
87  ->  81
88 

In [38]:
import pickle

# Save each matrix to a pickle file
with open('train_new.pickle', 'wb') as f:
    pickle.dump(train_set, f)

with open('valid_new.pickle', 'wb') as f:
    pickle.dump(valid_set, f)

with open('test_new.pickle', 'wb') as f:
    pickle.dump(test_set, f)

print(len(train_set))

237240
